## 함수 감싸기 

In [2]:
import time
from functools import wraps

def timethis(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        end = time.time()
        print(func.__name__, end-start)
        return result
    return wrapper

if __name__ == '__main__':
    @timethis
    def countdown(n:int):
        while n > 0:
            n -= 1

    countdown(100000)
    countdown(10000000)

countdown 0.004880189895629883
countdown 0.46359729766845703


## 함수 메타데이터 보존 

In [8]:
import time
from functools import wraps

def timethis(func):
    '''
    Decorator that reports the execution time.
    '''
    @wraps(func)
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        end = time.time()
        print(func.__name__, end-start)
        return result
    return wrapper

if __name__ == '__main__':
    @timethis
    def countdown(n:int):
        '''
        Counts down
        '''
        while n > 0:
            n -= 1

    countdown(100000)
    print('Name:', countdown.__name__)
    print('Docstring:', repr(countdown.__doc__))
    print('Annotations:', countdown.__annotations__)

countdown 0.004880428314208984
Name: countdown
Docstring: '\n        Counts down\n        '
Annotations: {'n': <class 'int'>}


## 데코레이터 풀기 

In [12]:
from functools import wraps

def decorator1(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        print('Decorator 1')
        return func(*args, **kwargs)
    return wrapper

def decorator2(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        print('Decorator 2')
        return func(*args, **kwargs)
    return wrapper

@decorator1
@decorator2
def add(x, y):
    return x + y

# Calling wrapped function
print(add(2,3))

# Calling original function
print(add.__wrapped__(2,3))

Decorator 1
Decorator 2
5
Decorator 2
5


## 매개변수를 받는 데코레이터 정의

In [13]:
from functools import wraps
import logging

def logged(level, name=None, message=None):
    '''
    Add logging to a function.  level is the logging
    level, name is the logger name, and message is the
    log message.  If name and message aren't specified,
    they default to the function's module and name.
    '''
    def decorate(func):
        logname = name if name else func.__module__
        log = logging.getLogger(logname)
        logmsg = message if message else func.__name__

        @wraps(func)
        def wrapper(*args, **kwargs):
            log.log(level, logmsg)
            return func(*args, **kwargs)
        return wrapper
    return decorate

# Example use
@logged(logging.DEBUG)
def add(x, y):
    return x + y

@logged(logging.CRITICAL, 'example')
def spam():
    print('Spam!')

if __name__ == '__main__':
    import logging
    logging.basicConfig(level=logging.DEBUG)
    print(add(2,3))
    spam()

DEBUG:__main__:add
CRITICAL:example:spam


5
Spam!


## 사용자가 조절 가능한 속성을 가진 데코레이터

In [14]:
from functools import wraps, partial
import logging

def attach_wrapper(obj, func=None):
    if func is None:
        return partial(attach_wrapper, obj)
    setattr(obj, func.__name__, func)
    return func

def logged(level, name=None, message=None):
    '''
    Add logging to a function.  level is the logging
    level, name is the logger name, and message is the
    log message.  If name and message aren't specified,
    they default to the function's module and name.
    '''
    def decorate(func):
        logname = name if name else func.__module__
        log = logging.getLogger(logname)
        logmsg = message if message else func.__name__

        @wraps(func)
        def wrapper(*args, **kwargs):
            log.log(level, logmsg)
            return func(*args, **kwargs)

        # Attach setter functions
        @attach_wrapper(wrapper)
        def set_level(newlevel):
            nonlocal level
            level = newlevel

        @attach_wrapper(wrapper)
        def set_message(newmsg):
            nonlocal logmsg
            logmsg = newmsg

        return wrapper
    return decorate

# Example use
@logged(logging.DEBUG)
def add(x, y):
    return x + y

@logged(logging.CRITICAL, 'example')
def spam():
    print('Spam!')

# Example involving multiple decorators

import time
def timethis(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        start = time.time()
        r = func(*args, **kwargs)
        end = time.time()
        print(func.__name__, end - start)
        return r
    return wrapper

@timethis
@logged(logging.DEBUG)
def countdown(n):
    while n > 0:
        n -= 1


@logged(logging.DEBUG)
@timethis
def countdown2(n):
    while n > 0:
        n -= 1

if __name__ == '__main__':
    import logging
    logging.basicConfig(level=logging.DEBUG)
    print(add(2, 3))

    # Change the log message
    add.set_message('Add called')
    print(add(2, 3))

    # Change the log level
    add.set_level(logging.WARNING)
    print(add(2, 3))

    countdown(100000)
    countdown.set_level(logging.CRITICAL)
    countdown(100000)

    countdown2(100000)
    countdown2.set_level(logging.CRITICAL)
    countdown2(100000)


DEBUG:__main__:add
DEBUG:__main__:Add called
DEBUG:__main__:countdown
CRITICAL:__main__:countdown
DEBUG:__main__:countdown2
CRITICAL:__main__:countdown2


5
5
5
countdown 0.004878997802734375
countdown 0.0058553218841552734
countdown2 0.004911661148071289
countdown2 0.0048792362213134766


## 옵션 매개변수를 받는 데코레이터

In [15]:
from functools import wraps, partial
import logging

def logged(func=None, *, level=logging.DEBUG, name=None, message=None):
    if func is None:
        return partial(logged, level=level, name=name, message=message)

    logname = name if name else func.__module__
    log = logging.getLogger(logname)
    logmsg = message if message else func.__name__
    @wraps(func)
    def wrapper(*args, **kwargs):
        log.log(level, logmsg)
        return func(*args, **kwargs)
    return wrapper

# Example use
@logged
def add(x, y):
    return x + y

@logged()
def sub(x, y):
    return x - y

@logged(level=logging.CRITICAL, name='example')
def spam():
    print('Spam!')

if __name__ == '__main__':
    import logging
    logging.basicConfig(level=logging.DEBUG)
    add(2,3)
    sub(2,3)
    spam()


DEBUG:__main__:add
DEBUG:__main__:sub
CRITICAL:example:spam


Spam!
